In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tf-roberta/pretrained-roberta-base.h5
/kaggle/input/tf-roberta/config-roberta-base.json
/kaggle/input/tf-roberta/vocab-roberta-base.json
/kaggle/input/tf-roberta/merges-roberta-base.txt


In [2]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version',tf.__version__)

TF version 2.6.2


In [3]:
MAX_LEN = 96
PATH = '../input/tf-roberta/'
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab=PATH+'vocab-roberta-base.json', 
    merges=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('../input/tweet-sentiment-extraction/train.csv').fillna('')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

In [5]:
for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[k,:len(enc.ids)+5] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

In [6]:
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1

In [7]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Conv1D(1,1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1,1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [8]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [9]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=777)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
#     #print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
#     print(fold)
#     print(idxT)
#     print(idxV)
#     continue
    sv = tf.keras.callbacks.ModelCheckpoint(
        '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')
        
    model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
        epochs=3, batch_size=32, verbose=DISPLAY, callbacks=[sv],
        validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
        [start_tokens[idxV,], end_tokens[idxV,]]))
    
    print('Loading model...')
    model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    
    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-1:b])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()
    

#########################
### FOLD 1
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
687/687 [==============================] - 288s 399ms/step - loss: 2.1879 - activation_loss: 1.0746 - activation_1_loss: 1.1133 - val_loss: 1.7671 - val_activation_loss: 0.8915 - val_activation_1_loss: 0.8756

Epoch 00001: val_loss improved from inf to 1.76708, saving model to v0-roberta-0.h5
Epoch 2/3
687/687 [==============================] - 272s 395ms/step - loss: 1.6850 - activation_loss: 0.8469 - activation_1_loss: 0.8381 - val_loss: 1.7049 - val_activation_loss: 0.8677 - val_activation_1_loss: 0.8372

Epoch 00002: val_loss improved from 1.76708 to 1.70489, saving model to v0-roberta-0.h5
Epoch 3/3
687/687 [==============================] - 271s 395ms/step - loss: 1.5325 - activation_loss: 0.7832 - activation_1_loss: 0.7493 - val_loss: 1.6855 - val_activation_loss: 0.8669 - val_activation_1_loss: 0.8186

Epoch 00003: val_loss improved from 1.70489 to 1.68548, saving model to v0-roberta-0.h5
Loading model...
Predicting OOF...
172/172 [==============================] - 26

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
688/688 [==============================] - 301s 417ms/step - loss: 2.2104 - activation_loss: 1.0945 - activation_1_loss: 1.1160 - val_loss: 1.7346 - val_activation_loss: 0.8763 - val_activation_1_loss: 0.8582

Epoch 00001: val_loss improved from inf to 1.73456, saving model to v0-roberta-1.h5
Epoch 2/3
688/688 [==============================] - 284s 413ms/step - loss: 1.6596 - activation_loss: 0.8469 - activation_1_loss: 0.8127 - val_loss: 1.6314 - val_activation_loss: 0.8372 - val_activation_1_loss: 0.7942

Epoch 00002: val_loss improved from 1.73456 to 1.63138, saving model to v0-roberta-1.h5
Epoch 3/3
688/688 [==============================] - 284s 413ms/step - loss: 1.5171 - activation_loss: 0.7839 - activation_1_loss: 0.7332 - val_loss: 1.6570 - val_activation_loss: 0.8470 - val_activation_1_loss: 0.8100

Epoch 00003: val_loss did not improve from 1.63138
Loading model...
Predicting OOF...
172/172 [==============================] - 26s 136ms/step
Predicting Test...
111/1

...

Epoch 1/3
688/688 [==============================] - 301s 418ms/step - loss: 2.1510 - activation_loss: 1.0656 - activation_1_loss: 1.0855 - val_loss: 1.6867 - val_activation_loss: 0.8623 - val_activation_1_loss: 0.8244

Epoch 00001: val_loss improved from inf to 1.68669, saving model to v0-roberta-2.h5
Epoch 2/3
688/688 [==============================] - 285s 414ms/step - loss: 1.6747 - activation_loss: 0.8507 - activation_1_loss: 0.8240 - val_loss: 1.6449 - val_activation_loss: 0.8301 - val_activation_1_loss: 0.8148

Epoch 00002: val_loss improved from 1.68669 to 1.64487, saving model to v0-roberta-2.h5
Epoch 3/3
688/688 [==============================] - 284s 413ms/step - loss: 1.5713 - activation_loss: 0.7907 - activation_1_loss: 0.7806 - val_loss: 1.6777 - val_activation_loss: 0.8368 - val_activation_1_loss: 0.8409

Epoch 00003: val_loss did not improve from 1.64487
Loading model...
Predicting OOF...
172/172 [==============================] - 26s 136ms/step
Predicting Test...
111/1

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
688/688 [==============================] - 302s 418ms/step - loss: 2.0907 - activation_loss: 1.0508 - activation_1_loss: 1.0399 - val_loss: 1.6661 - val_activation_loss: 0.8509 - val_activation_1_loss: 0.8152

Epoch 00001: val_loss improved from inf to 1.66611, saving model to v0-roberta-3.h5
Epoch 2/3
688/688 [==============================] - 286s 415ms/step - loss: 1.6158 - activation_loss: 0.8315 - activation_1_loss: 0.7843 - val_loss: 1.5930 - val_activation_loss: 0.8112 - val_activation_1_loss: 0.7817

Epoch 00002: val_loss improved from 1.66611 to 1.59298, saving model to v0-roberta-3.h5
Epoch 3/3
688/688 [==============================] - 287s 417ms/step - loss: 1.4619 - activation_loss: 0.7567 - activation_1_loss: 0.7052 - val_loss: 1.6332 - val_activation_loss: 0.8178 - val_activation_1_loss: 0.8154

Epoch 00003: val_loss did not improve from 1.59298
Loading model...
Predicting OOF...
172/172 [==============================] - 28s 139ms/step
Predicting Test...
111/1

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at ../input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
688/688 [==============================] - 308s 424ms/step - loss: 2.1919 - activation_loss: 1.0913 - activation_1_loss: 1.1005 - val_loss: 1.6687 - val_activation_loss: 0.8523 - val_activation_1_loss: 0.8165

Epoch 00001: val_loss improved from inf to 1.66871, saving model to v0-roberta-4.h5
Epoch 2/3
688/688 [==============================] - 289s 420ms/step - loss: 1.6575 - activation_loss: 0.8537 - activation_1_loss: 0.8038 - val_loss: 1.6335 - val_activation_loss: 0.8396 - val_activation_1_loss: 0.7939

Epoch 00002: val_loss improved from 1.66871 to 1.63346, saving model to v0-roberta-4.h5
Epoch 3/3
688/688 [==============================] - 288s 419ms/step - loss: 1.5062 - activation_loss: 0.7751 - activation_1_loss: 0.7312 - val_loss: 1.6175 - val_activation_loss: 0.8380 - val_activation_1_loss: 0.7794

Epoch 00003: val_loss improved from 1.63346 to 1.61747, saving model to v0-roberta-4.h5
Loading model...
Predicting OOF...
172/172 [==============================] - 29

In [10]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-1:b])
    all.append(st)

In [11]:
test['selected_text'] = all
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

,textID,text,sentiment,selected_text
769,0117a03b6d,_43 HAHHAHAH that really made me laugh out loudd! ahahah...,positive,laugh out loudd! ahahahahahahahahah your fuunny!!
3326,6face8f81f,"I can`t, I have an exam on wednesday",neutral,"i can`t, i have an exam on wednesday"
480,819626535b,"Glad to hear you made it out, I hear that place used to...",positive,glad
2113,ebf2a10961,I KNO!!! im so sad! evry1 is leavin its horrible! im su...,negative,sad!
2036,1bed6bec2a,Finally finished exams & home not watch Otalia & the PBP...,neutral,finally finished exams & home not watch otalia & the pb...
3092,3ed1480471,wonders if anyone would care if she died tomorrow,negative,wonders if anyone would care if she died tomorrow
3423,1d865c0717,I have the biggest headache ever. My photosensitivity is...,negative,i have the biggest headache ever.
1973,32b129dc3f,says Happy Mother`s Day to all the Moms out there.,positive,happy
2290,c9a52dee1f,"Guess I`m gonna try the nap thing again 2day, but since ...",negative,since my kids haven`t cooperated with it yet this week ...
2757,a4468f0e74,Gotcha ! Let`s meet up @ The Twitt Cafe,neutral,gotcha ! let`s meet up @ the twitt cafe
